# AEP Dynamic Grid Challenge - Starting from Scratch

Let's build this step by step, starting with understanding our data.

In [1]:
# Import essential libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Step 1: Load and Understand the Data

Our first method will load all the key datasets and give us a clear picture of what we're working with.

In [2]:
def load_and_explore_data():
    """Load all datasets and provide comprehensive overview of the Hawaii 40-bus system."""
    
    print("🔌 AEP DYNAMIC GRID CHALLENGE - DATA EXPLORATION")
    print("=" * 60)
    
    # Load core datasets
    print("📂 Loading datasets...")
    
    try:
        # Power system model data
        lines_df = pd.read_csv('hawaii40_osu/csv/lines.csv')
        buses_df = pd.read_csv('hawaii40_osu/csv/buses.csv')
        generators_df = pd.read_csv('hawaii40_osu/csv/generators.csv')
        loads_df = pd.read_csv('hawaii40_osu/csv/loads.csv')
        
        # Flow data
        flows_df = pd.read_csv('hawaii40_osu/line_flows_nominal.csv')
        
        # Conductor library for IEEE 738 calculations
        conductor_df = pd.read_csv('ieee738/conductor_library.csv')
        
        print("✅ All datasets loaded successfully!")
        
    except FileNotFoundError as e:
        print(f"❌ Error loading data: {e}")
        print("Make sure you're running from the project root directory.")
        return None
    
    print("\n📊 SYSTEM OVERVIEW")
    print("-" * 30)
    print(f"🏢 Buses (substations): {len(buses_df)}")
    print(f"⚡ Transmission lines: {len(lines_df)}")
    print(f"🔋 Generators: {len(generators_df)}")
    print(f"🏭 Loads: {len(loads_df)}")
    print(f"🔌 Conductor types: {len(conductor_df)}")
    
    # Voltage level analysis
    voltage_levels = sorted(buses_df['v_nom'].unique())
    print(f"⚡ Voltage levels: {voltage_levels} kV")
    
    # Merge lines with flows to understand current loading
    print("\n🔄 CURRENT SYSTEM LOADING")
    print("-" * 30)
    
    # Merge line data with flow data
    system_data = lines_df.merge(flows_df, on='name', how='left')
    
    # Calculate loading percentages
    system_data['loading_pct'] = (system_data['p0_nominal'] / system_data['s_nom']) * 100
    
    # System loading statistics
    max_loading = system_data['loading_pct'].max()
    avg_loading = system_data['loading_pct'].mean()
    lines_over_50 = len(system_data[system_data['loading_pct'] > 50])
    lines_over_75 = len(system_data[system_data['loading_pct'] > 75])
    
    print(f"📈 Maximum loading: {max_loading:.1f}%")
    print(f"📊 Average loading: {avg_loading:.1f}%")
    print(f"⚠️  Lines >50% loaded: {lines_over_50}")
    print(f"🚨 Lines >75% loaded: {lines_over_75}")
    
    # Show top loaded lines
    print("\n🔥 TOP 10 MOST LOADED LINES")
    print("-" * 50)
    top_loaded = system_data.nlargest(10, 'loading_pct')[[
        'name', 'branch_name', 'loading_pct', 'conductor', 's_nom', 'p0_nominal'
    ]].copy()
    
    # Format for display
    top_loaded['loading_pct'] = top_loaded['loading_pct'].round(1)
    top_loaded['s_nom'] = top_loaded['s_nom'].round(0)
    top_loaded['p0_nominal'] = top_loaded['p0_nominal'].round(1)
    
    print(top_loaded.to_string(index=False))
    
    # Conductor analysis
    print("\n🔌 CONDUCTOR ANALYSIS")
    print("-" * 25)
    conductor_usage = system_data['conductor'].value_counts()
    print("Most common conductors:")
    for conductor, count in conductor_usage.head(5).items():
        print(f"  {conductor}: {count} lines")
    
    # Show available conductor properties
    print("\n📋 Available conductor properties:")
    print(f"  Columns: {list(conductor_df.columns)}")
    print(f"  Sample conductor data:")
    print(conductor_df.head(3).to_string(index=False))
    
    # Voltage level breakdown
    print("\n⚡ VOLTAGE LEVEL BREAKDOWN")
    print("-" * 30)
    
    # Merge with bus data to get voltage levels
    system_with_voltage = system_data.merge(
        buses_df[['name', 'v_nom']], 
        left_on='bus0', 
        right_on='name', 
        suffixes=('', '_bus')
    )
    
    voltage_summary = system_with_voltage.groupby('v_nom').agg({
        'name': 'count',
        'loading_pct': ['mean', 'max'],
        's_nom': 'mean'
    }).round(1)
    
    voltage_summary.columns = ['Line_Count', 'Avg_Loading_%', 'Max_Loading_%', 'Avg_Rating_MVA']
    print(voltage_summary)
    
    print("\n✅ DATA EXPLORATION COMPLETE!")
    print("\n💡 KEY INSIGHTS:")
    print(f"   • System is lightly loaded (max {max_loading:.1f}%)")
    print(f"   • {len(voltage_levels)} voltage levels: {voltage_levels} kV")
    print(f"   • {len(conductor_df)} different conductor types available")
    print(f"   • Ready for weather impact analysis!")
    
    # Return datasets for further analysis
    return {
        'lines': lines_df,
        'buses': buses_df,
        'generators': generators_df,
        'loads': loads_df,
        'flows': flows_df,
        'conductors': conductor_df,
        'system_data': system_data
    }

# Run the exploration
data = load_and_explore_data()

🔌 AEP DYNAMIC GRID CHALLENGE - DATA EXPLORATION
📂 Loading datasets...
✅ All datasets loaded successfully!

📊 SYSTEM OVERVIEW
------------------------------
🏢 Buses (substations): 37
⚡ Transmission lines: 77
🔋 Generators: 45
🏭 Loads: 27
🔌 Conductor types: 8
⚡ Voltage levels: [69.0, 138.0] kV

🔄 CURRENT SYSTEM LOADING
------------------------------
📈 Maximum loading: 80.8%
📊 Average loading: 39.1%
⚠️  Lines >50% loaded: 29
🚨 Lines >75% loaded: 5

🔥 TOP 10 MOST LOADED LINES
--------------------------------------------------
name                                 branch_name  loading_pct              conductor  s_nom  p0_nominal
 L48 PEARL CITY69 (19)  TO  WAIPAHU69 (23) CKT 1         80.8   556.5 ACSR 26/7 DOVE     96        77.5
 L49 PEARL CITY69 (19)  TO  WAIPAHU69 (23) CKT 2         80.8   556.5 ACSR 26/7 DOVE     96        77.5
 L50 PEARL CITY69 (19)  TO  WAIPAHU69 (23) CKT 3         80.8   556.5 ACSR 26/7 DOVE     96        77.5
 L51 PEARL CITY69 (19)  TO  WAIPAHU69 (23) CKT 4         

## Quick Visualization

Let's create a simple visualization to see the current system loading.

In [ ]:
if data is not None:
    # Create a simple loading distribution plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Loading distribution histogram
    ax1.hist(data['system_data']['loading_pct'], bins=20, alpha=0.7, edgecolor='black')
    ax1.axvline(x=50, color='orange', linestyle='--', label='50% Caution')
    ax1.axvline(x=75, color='red', linestyle='--', label='75% Warning')
    ax1.set_xlabel('Loading (%)')
    ax1.set_ylabel('Number of Lines')
    ax1.set_title('Current Line Loading Distribution')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Top 15 loaded lines
    top_15 = data['system_data'].nlargest(15, 'loading_pct')
    colors = ['red' if x > 75 else 'orange' if x > 50 else 'green' for x in top_15['loading_pct']]
    
    ax2.barh(range(len(top_15)), top_15['loading_pct'], color=colors, alpha=0.7)
    ax2.set_yticks(range(len(top_15)))
    ax2.set_yticklabels([name[:20] + '...' if len(name) > 20 else name for name in top_15['name']], fontsize=8)
    ax2.set_xlabel('Loading (%)')
    ax2.set_title('Top 15 Most Loaded Lines')
    ax2.axvline(x=50, color='orange', linestyle='--', alpha=0.5)
    ax2.axvline(x=75, color='red', linestyle='--', alpha=0.5)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Current system shows light loading - perfect for weather stress testing!")
else:
    print("❌ Data not loaded - check file paths")

## Next Steps

Now that we understand our baseline data, we can move to:

1. **IEEE 738 Implementation** - Calculate dynamic ratings based on weather
2. **Temperature Sensitivity Analysis** - Find when lines start overloading
3. **Visualization Tools** - Create interactive dashboards
4. **N-1 Contingency Analysis** - Bonus challenge implementation

**Key Findings from this exploration:**
- System is currently lightly loaded (good for stress testing)
- Multiple voltage levels and conductor types
- Ready to implement weather-based dynamic rating calculations